In [1]:
# !pip install tensorflow
# !pip install Keras-Preprocessing


In [2]:
import os
import pandas as pd
import numpy as np

from pyvi.ViTokenizer import ViTokenizer #vietnamese tokenizer

import re #regular expression

In [3]:
import pickle
from keras_preprocessing.sequence import pad_sequences

# from tensorflow.keras.models import Sequential 

from tensorflow.keras import layers, Sequential

In [4]:
import tensorflow as tf
# from keras.preprocessing import text, sequence, tokenizer

from tensorflow.keras.preprocessing import text, sequence 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Conv2D, MaxPool2D, Bidirectional, LSTM, GRU, concatenate, GlobalMaxPooling1D, GlobalAveragePooling1D, SpatialDropout1D, Reshape, Dropout, Concatenate
#from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

from keras.layers import Input, Dense, Embedding, Flatten, Conv2D, MaxPool2D, Bidirectional, LSTM, GRU, concatenate, GlobalMaxPooling1D, GlobalAveragePooling1D, SpatialDropout1D, Reshape, Flatten, Dropout, Concatenate

In [5]:
DIR_DATASET = os.path.join('dataset')

PATH_TRAIN = os.path.join(DIR_DATASET, 'core-data\\train.csv')
PATH_DEV = os.path.join(DIR_DATASET, 'core-data\dev.csv')
PATH_TEST = os.path.join(DIR_DATASET, 'core-data\\test.csv')

EMBEDDING_PATH = os.path.join(DIR_DATASET, 'cc.vi.300.vec')
TOKENIZER_PATH = os.path.join(DIR_DATASET, 'tokenizer.pickle')
VISTOPWORDS_PATH = os.path.join(DIR_DATASET, "vietnamese-stopwords.txt")

MODEL_DIR = 'model'
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

In [6]:
data_train = pd.read_csv(PATH_TRAIN)

data_dev = pd.read_csv(PATH_DEV)

data_test = pd.read_csv(PATH_TEST)

In [7]:
with open(VISTOPWORDS_PATH, "r", encoding="utf-8") as ins:
    stopwords = []
    for line in ins:
        dd = line.strip('\n')
        stopwords.append(dd)
    stopwords = set(stopwords)

In [8]:
# read csv data
data_train = pd.read_csv(PATH_TRAIN)
data_dev = pd.read_csv(PATH_DEV)
data_test = pd.read_csv(PATH_TEST)

In [9]:
data_train.columns

Index(['link', 'rating', 'comment', 'categories', 'category', 'product_name',
       'description', 'num_sold', 'num_review', 'label', 'spam_label'],
      dtype='object')

In [10]:
data_train=data_train.drop(['link', 'rating', 'categories', 'category', 'product_name',
       'description', 'num_sold', 'num_review', 'spam_label'], axis=1)
data_dev=data_dev.drop(['link', 'rating', 'categories', 'category', 'product_name',
       'description', 'num_sold', 'num_review', 'spam_label'], axis=1)
data_test=data_test.drop(['link', 'rating', 'categories', 'category', 'product_name',
       'description', 'num_sold', 'num_review', 'spam_label'], axis=1)

In [11]:
data_train.head(10)

,comment,label
0,sản phẩm đúng quảng cáo. giao hàng hơi chậm nh...,0
1,"Mình nhận hàng ngày hôm qua, tiki giao hàng đú...",0
2,"Máy giặt quá tốt, chỉ có 5 tr được cái máy giặ...",0
3,chất liệu vải tốt mát nói chung là chất lượng ...,0
4,Tủ tốt giá tốt nhiều hơn so với các trung tâm...,0
5,Mọi người dưới 56kg nên mua sz S thoii nha. Fo...,1
6,giao nhanh đóng hàng cẩn thận ăn ngon lắm,0
7,10 điểm. Giao hàng nhanh. Cầu hình cao màn hìn...,0
8,Bé nhà mình tiêu thụ loại này tốt!! Tuy nhiên ...,0
9,Ỏ Bến Tre mà vừa đặt hàng hôm qua nay nhận dc ...,0


In [12]:
dup_train = data_train.duplicated().sum()
dup_dev = data_dev.duplicated().sum()
dup_test = data_test.duplicated().sum()

print("train's duplicates: ", dup_train,
    "\ndev's duplicates: ", dup_dev, "\ntest's duplicates: ", dup_test)

train's duplicates:  8 
dev's duplicates:  0 
test's duplicates:  3


In [13]:
data_train=data_train.drop_duplicates(keep='first')
data_dev=data_dev.drop_duplicates(keep='first')
data_test=data_test.drop_duplicates(keep='first')

In [14]:
dup_train = data_train.duplicated().sum()
dup_dev = data_dev.duplicated().sum()
dup_test = data_test.duplicated().sum()

print("train's duplicates: ", dup_train,
    "\ndev's duplicates: ", dup_dev, "\ntest's duplicates: ", dup_test)

train's duplicates:  0 
dev's duplicates:  0 
test's duplicates:  0


PREPROCESS FUNCTION

In [15]:
# filder stop word
def filter_stop_words(train_sentences, stop_words):
    new_sent = [word for word in train_sentences.split() if word not in stop_words]
    train_sentences = ' '.join(new_sent)
    return train_sentences

# remove emoji
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)
# declare func and adding some remove
def preprocess(text, tokenized = True, lowercased = True):
    text = ViTokenizer.tokenize(text) if tokenized else text
    text = filter_stop_words(text, stopwords)
    text = deEmojify(text)
    text = text.lower() if lowercased else text

    text = text.strip()
    text = re.compile('<.*?>').sub('', text)
    text = re.sub('\s+', ' ', text) 
    text = re.sub(r'\[[0-9]*\]',' ',text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d',' ',text)
    text = re.sub(r'\s+',' ',text)
    return text

def pre_process_features(X, y, tokenized = True, lowercased = True):
    X = np.array(X)
    y = np.array(y)
    X = [preprocess(str(p), tokenized = tokenized, lowercased = lowercased) for p in list(X)]
    for idx, ele in enumerate(X):
        if not ele:
            X = np.delete(X, idx)
            y = np.delete(y, idx)
    return X, y


In [16]:
text_01 = "[HN] Hiện tại bên mình vẫn đang cần tuyển vị trí C# developer (.NET framework, C# biết winforms (2-3 năm)) sẵn sàng làm thêm FE hoặc Python, Java nếu được giao. Lương thỏa thuận, có 13-15 tháng lương. Quan tâm ib lấy JD."
text_02 = preprocess(text_01)
print("After preprocess: ", text_01, "\n")
print("Before preprocess: ", text_02)

After preprocess:  [HN] Hiện tại bên mình vẫn đang cần tuyển vị trí C# developer (.NET framework, C# biết winforms (2-3 năm)) sẵn sàng làm thêm FE hoặc Python, Java nếu được giao. Lương thỏa thuận, có 13-15 tháng lương. Quan tâm ib lấy JD. 

Before preprocess:   hn hiện_tại tuyển vị_trí c developer net framework c winforms sẵn_sàng fe python java giao lương thỏa_thuận lương quan_tâm ib jd


PREPROCESS

In [17]:
data_train

,comment,label
0,sản phẩm đúng quảng cáo. giao hàng hơi chậm nh...,0
1,"Mình nhận hàng ngày hôm qua, tiki giao hàng đú...",0
2,"Máy giặt quá tốt, chỉ có 5 tr được cái máy giặ...",0
3,chất liệu vải tốt mát nói chung là chất lượng ...,0
4,Tủ tốt giá tốt nhiều hơn so với các trung tâm...,0
...,...,...
14301,dùng okee ko gây nóng máy nhưng chất của ốp om...,0
14302,"Rất tốt, rất đáng tiền, nằm rất êm, lúc mới mu...",0
14303,"giao đủ, đóng gói chắc chắn, giao hàng nhanh \...",1
14304,giao hỏa tốc rẻ và nhanh. Nhưng trước đó mình ...,0


In [18]:
# slit data so that X_train is review and y_train is label
X_train = data_train.iloc[:, 0:1]
y_train = data_train.iloc[:, 1:2]

X_dev = data_train.iloc[:, 0:1]
y_dev = data_train.iloc[:, 1:2]

X_test = data_train.iloc[:, 0:1]
y_test = data_train.iloc[:, 1:2]    

In [19]:
y_train

,label
0,0
1,0
2,0
3,0
4,0
...,...
14301,0
14302,0
14303,1
14304,0


In [20]:
# transform X_train into narray (train_X) and simular
train_X, train_y= pre_process_features(X_train['comment'], y_train['label'], tokenized=True, lowercased = True)
dev_X, dev_y= pre_process_features(X_dev['comment'], y_dev['label'], tokenized=True, lowercased = True)
test_X, test_y = pre_process_features(X_test['comment'], y_test['label'], tokenized=True, lowercased = True)

In [21]:
train_X.shape

(14271,)

In [22]:
train_y.shape

(14271,)

DECLARE FUNCTIONS

In [23]:
embeddings_index = {}
with open(EMBEDDING_PATH, encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [24]:
tokenizer = text.Tokenizer(lower=False, filters='!"#$%&()*+,-./:;<=>?@[\]^`{|}~\t\n')
tokenizer.fit_on_texts(train_X)

In [25]:
with open(TOKENIZER_PATH, 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [26]:
# def make_featues(X, y, tokenizer, is_one_hot_label=True, number_class1=2):
#     X = tokenizer.texts_to_sequences(X)
#     X = pad_sequences(X, maxlen=sequence_length)
#     if is_one_hot_label: 
#         y = to_categorical(y, num_classes=number_class1)
#     return X, y

def make_featues(X, y, tokenizer, is_one_hot_label=False, number_class1=2): # my model just binary (non-spam or not spam)
    X = tokenizer.texts_to_sequences(X)
    X = pad_sequences(X, maxlen=sequence_length)
    if is_one_hot_label: 
        y = to_categorical(y, num_classes=number_class1)
    return X, y

In [27]:
vocabulary_size = 10000
sequence_length = 100
maxlen = 100

embedding_dim = 300
batch_size = 256
epochs = 40
drop = 0.5

# filter_sizes = [2,3,5]
# num_filters = 32

filter_sizes = [2,3,5]
num_filters = 32

word_index = tokenizer.word_index
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i >= vocabulary_size:
        continue

    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [28]:
vocabulary_size = 10000
sequence_length = 100

embedding_dim = 300
batch_size = 256
epochs = 40
drop = 0.5

filter_sizes = [2,3,5]
num_filters = 32

word_index = tokenizer.word_index
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i >= vocabulary_size:
        continue

    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [29]:
train_XX, train_yy = make_featues(train_X, train_y, tokenizer)
dev_XX, dev_yy = make_featues(dev_X, dev_y, tokenizer)
test_XX, test_yy = make_featues(test_X, test_y, tokenizer)

In [30]:
train_yy

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [31]:
train_XX.shape

(14271, 100)

In [32]:
print(train_XX[5, :])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0  268  141    3  710 6622   18
  347   49  414  497   13   65   82 6623    4    7  711   37  119 2741
  498  831]


In [33]:
print(type(train_XX), train_XX.shape)
print(type(train_yy), train_yy.shape)
print(type(dev_XX), dev_XX.shape)
print(type(dev_yy), dev_yy.shape)
print(type(test_XX), test_XX.shape)
print(type(test_yy), test_yy.shape)

<class 'numpy.ndarray'> (14271, 100)
<class 'numpy.ndarray'> (14271,)
<class 'numpy.ndarray'> (14271, 100)
<class 'numpy.ndarray'> (14271,)
<class 'numpy.ndarray'> (14271, 100)
<class 'numpy.ndarray'> (14271,)


In [34]:
train_XX.shape

(14271, 100)

CNNs MODEL

In [35]:
# TextCNN
# init layers
textCNNModel = Sequential()

# emmbed word vector
textCNNModel.add(layers.Embedding(num_words, embedding_dim, input_length=sequence_length))

# model
textCNNModel.add(layers.Conv1D(128, 5, activation='relu'))
textCNNModel.add(layers.GlobalMaxPooling1D())
textCNNModel.add(layers.Dense(256, activation='relu'))

# Output layer for binary classification
textCNNModel.add(layers.Dense(1, activation='sigmoid')) 

# Compile the model
textCNNModel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 

textCNNModel.summary()

E:\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ ?                           │     0 (unbuilt) │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [36]:
# Test for model accuracy
textCNNModel.fit( train_XX, train_yy,
    epochs=50,
    verbose=2,
    validation_data=(dev_XX, dev_yy),
    batch_size=batch_size)

Epoch 1/50
56/56 - 13s - 224ms/step - accuracy: 0.7557 - loss: 0.5069 - val_accuracy: 0.8598 - val_loss: 0.3584
Epoch 2/50
56/56 - 11s - 192ms/step - accuracy: 0.8683 - loss: 0.3258 - val_accuracy: 0.9248 - val_loss: 0.2096
Epoch 3/50
56/56 - 12s - 206ms/step - accuracy: 0.9241 - loss: 0.1993 - val_accuracy: 0.9615 - val_loss: 0.1211
Epoch 4/50
56/56 - 11s - 198ms/step - accuracy: 0.9638 - loss: 0.1055 - val_accuracy: 0.9813 - val_loss: 0.0553
Epoch 5/50
56/56 - 11s - 196ms/step - accuracy: 0.9813 - loss: 0.0543 - val_accuracy: 0.9914 - val_loss: 0.0306
Epoch 6/50
56/56 - 11s - 199ms/step - accuracy: 0.9890 - loss: 0.0335 - val_accuracy: 0.9940 - val_loss: 0.0213
Epoch 7/50
56/56 - 12s - 216ms/step - accuracy: 0.9917 - loss: 0.0248 - val_accuracy: 0.9949 - val_loss: 0.0174
Epoch 8/50
56/56 - 11s - 199ms/step - accuracy: 0.9940 - loss: 0.0185 - val_accuracy: 0.9954 - val_loss: 0.0137
Epoch 9/50
56/56 - 11s - 200ms/step - accuracy: 0.9941 - loss: 0.0174 - val_accuracy: 0.9941 - val_loss:

In [37]:
# Evaluate the model on the TEST data
loss, accuracy = textCNNModel.evaluate(test_XX, test_yy)
print(f"LOSS: {loss}")
print ('Accuracy '+ str(accuracy))

446/446 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9986 - loss: 0.0036
LOSS: 0.00463847117498517
Accuracy 0.9980379939079285


LSTM MODEL

In [38]:
# LSTM
# init layer
lstmModel = Sequential()

# emmbed word vectors
lstmModel.add( layers.Embedding( num_words, embedding_dim, input_length=sequence_length ) )
lstmModel.add( layers.SpatialDropout1D(0.2) )

# lstmModel.add( layers.LSTM( 100, return_sequences=False, dropout=0.5, recurrent_dropout=0.5 ) )
lstmModel.add( layers.LSTM( 100, return_sequences=False ) )
lstmModel.add( layers.Dense( 512, activation="relu" ) )

# Output layer for binary classification
lstmModel.add( layers.Dense( 1, activation="sigmoid" ) )

# Compile the model
lstmModel.compile( optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

lstmModel.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d (SpatialDropout1D) │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [39]:
# utils.plot_model(lstmModel)
lstmModel.fit( train_XX, train_yy,
    epochs=50,
    verbose=2,
    validation_data=(test_XX, test_yy),
    batch_size=batch_size)

Epoch 1/50
56/56 - 26s - 470ms/step - accuracy: 0.7800 - loss: 0.4932 - val_accuracy: 0.8599 - val_loss: 0.3359
Epoch 2/50
56/56 - 22s - 384ms/step - accuracy: 0.8629 - loss: 0.3265 - val_accuracy: 0.9074 - val_loss: 0.2506
Epoch 3/50
56/56 - 22s - 386ms/step - accuracy: 0.9060 - loss: 0.2433 - val_accuracy: 0.9287 - val_loss: 0.1989
Epoch 4/50
56/56 - 21s - 370ms/step - accuracy: 0.9292 - loss: 0.1848 - val_accuracy: 0.9536 - val_loss: 0.1290
Epoch 5/50
56/56 - 22s - 394ms/step - accuracy: 0.9474 - loss: 0.1413 - val_accuracy: 0.9603 - val_loss: 0.1091
Epoch 6/50
56/56 - 20s - 361ms/step - accuracy: 0.9566 - loss: 0.1172 - val_accuracy: 0.9676 - val_loss: 0.0864
Epoch 7/50
56/56 - 23s - 404ms/step - accuracy: 0.9621 - loss: 0.1003 - val_accuracy: 0.9701 - val_loss: 0.0868
Epoch 8/50
56/56 - 20s - 363ms/step - accuracy: 0.9654 - loss: 0.0892 - val_accuracy: 0.9690 - val_loss: 0.0750
Epoch 9/50
56/56 - 20s - 360ms/step - accuracy: 0.9702 - loss: 0.0769 - val_accuracy: 0.9777 - val_loss:

In [135]:
# Evaluate the model on the DEV data
loss, accuracy = lstmModel.evaluate(test_XX, test_yy)
print(f"LOSS: {loss}")
print ('Accuracy '+ str(accuracy))

446/446 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.9980 - loss: 0.0073
LOSS: 0.008859078399837017
Accuracy 0.9969869256019592


SAVE MODEL

In [175]:
CNNs = os.path.join(MODEL_DIR, 'CNNs.keras')  # Use .keras extension

# Save the model
textCNNModel.save(CNNs)

In [176]:
LSTM = os.path.join(MODEL_DIR, 'LSTM.keras')  # Use .keras extension

# Save the model
lstmModel.save(LSTM)


USE MODEL

In [163]:
input_string0 = "Bỉm dùng thích, mua sale được giá tốt shop giao hàng nhanh sẽ ủng hộ tiếp"
input_string1 = "Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu :)))) Xu"
input_string2 = "Âjskskxjxndndjsjdjskssosojsjsisossksksjdhhsjsussusjsjsjsjs"
input_string4 = "Có 4 Màu : Bạc - Vàng - Đen - Vàng Hồng ✚ Sản Phẩm : WR ✚ Tình trạng : mới ✚ Đồng hồ : nam nữ ✚ Kích thước: 36mm Dày : 5mm ✚ Chất liệu khung đồng hồ: thép không gỉ ✚ Chất liệu kính: Chống xước tốt. ✚ Loại dây đeo: thép không gỉ ✚ Chức năng hiển thị: Giờ, Phút, Giây, ngày, tháng ✚ Năng lượng: dùng pin ✚ Khả năng chống nước: chống nước sinh hoạt ✚ Bảo hành: 6 tháng ➥ Cam Kết ★★ 🚗 GIAO HÀNG & THANH TOÁN (COD) TẬN NƠI TRÊN TOÀN QUỐC. 👆 👆 BẢO HÀNH MÁY 6 THÁNG, PIN 12 THÁNG, 1 ĐỔI 1 TRONG VÒNG 7 NGÀY NẾU SẢN PHẨM LÀ LỖI CỦA NHÀ SẢN XUẤT 💲 KHÔNG BẢO HÀNH TRẦY XƯỚC BÊN NGOÀI #dongho #đồnghồnữ #sale #sangtrong #donghosapphire #donghothoitrang #donghonu #thoitrang #donghocaocap #donghogiare #donghodoi #đồnghồ #đồnghồnam #donghonam #chongnuoc"

tokenizer = text.Tokenizer(lower=False, filters='!"#$%&()*+,-./:;<=>?@[\]^`{|}~\t\n')
tokenizer.fit_on_texts([input_string4])
input_sequence = tokenizer.texts_to_sequences([input_string4])
padded_input_sequence = pad_sequences(input_sequence, maxlen=maxlen) 

predictions = textCNNModel.predict(padded_input_sequence)

# Assuming binary classification (0 or 1)
predicted_class = int(predictions > 0.5)  # assuming threshold of 0.5 for binary classification

print("padding string ", padded_input_sequence)
print("predict score: ", predictions)

if predicted_class > 0.5:
    print("type: spam")
else:
    print("type: non-spam")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
padding string  [[  1  46  47  48   6   7   8   1  49   9  50  51  52  53  54  55  10   1
   56  57  58  59   1  60   9  11  12  11  12  61  62   1  63  64  13  10
   65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  14  14  15
   16  80  13  17  81  82  17  18  83  18  84  85  86  87  88  19  89  90
   91  92  93  19  94  95  96  15  16  97  98  99 100 101 102 103 104 105
  106 107 108 109 110 111 112 113 114 115]]
predict score:  [[4.3783963e-08]]
type: non-spam


C:\Users\anhthy\AppData\Local\Temp\ipykernel_448\1473920951.py:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_class = int(predictions > 0.5)  # assuming threshold of 0.5 for binary classification


test in df

In [123]:
PATH_CRAWL= os.path.join(DIR_DATASET, 'core-data\\crawl.csv')
data_crawl = pd.read_csv(PATH_CRAWL)


In [124]:
data_crawl

,Username,Rating,Posted At,Product Categories,Review Text,Shop's response,Likes,Page
0,h*****a,5,2021-10-26 10:34,NaN,"Hàng giao đúng như trong mô tả, đóng gói đẹp, ...",NaN,0,1
1,ngocshina,5,2022-04-26 16:29,NaN,Chưa dùng nên chưa biết hiệu quả. Giao hàng nh...,NaN,0,1
2,t*****9,5,2022-02-16 21:19,NaN,"Sản phẩm giao hàng rất nhanh, chất lượng tốt c...",NaN,0,1
3,hong_minh79,5,2023-02-14 22:21,NaN,Công dụng: giải rượu\nMình đã mua lần 2. Uống ...,"Rohto Mentholatum Việt Nam xin chào bạn, Rohto...",0,1
4,trangtranggggg144,5,2021-10-22 09:32,NaN,"Hàng ok, đóng gói cẩm thận k bị vỡ. Chất lượng...",NaN,0,1
...,...,...,...,...,...,...,...,...
247,hoangtuga,3,2023-05-17 21:40,NaN,NaN,Rohto Mentholatum Việt Nam xin chào bạn ạ Roht...,0,1
248,tri_t_tien,2,2021-07-22 16:56,NaN,NaN,NaN,0,1
249,minhminh070398,1,2021-05-19 17:20,NaN,Shop bảo có quà mà lại Không có quà tặng đi kèm,NaN,0,1
250,tuantuoity_bn.84,1,2022-04-17 11:04,NaN,NaN,"Cảm ơn bạn đã lựa chọn sản phẩm, shop rất mong...",0,1


In [128]:
crawl_X = data_crawl.iloc[:, 4:5]
# y_train = data_train.iloc[:, 1:2]
crawl_X = crawl_X.dropna()
crawl_X

,Review Text
0,"Hàng giao đúng như trong mô tả, đóng gói đẹp, ..."
1,Chưa dùng nên chưa biết hiệu quả. Giao hàng nh...
2,"Sản phẩm giao hàng rất nhanh, chất lượng tốt c..."
3,Công dụng: giải rượu\nMình đã mua lần 2. Uống ...
4,"Hàng ok, đóng gói cẩm thận k bị vỡ. Chất lượng..."
...,...
242,"Sản phẩm tốt, chất lượng tuy nhiên gói hàng sơ..."
243,San pham su dung tot nen mua them cho gia dinh...
245,"Đóng gói kĩ, giao hàng nhanh như mọi khi nhưng..."
246,Hết tặng đèn thì gỡ xuống đi cho khách người t...


In [129]:
def pre_process_comment_array(X, tokenized = True, lowercased = True):
    X = np.array(X)
    X = [preprocess(str(p), tokenized = tokenized, lowercased = lowercased) for p in list(X)]
    for idx, ele in enumerate(X):
        if not ele:
            X = np.delete(X, idx)
    return X

def make_featues_comment_array(X, tokenizer, is_one_hot_label=False, number_class1=2):
    X = tokenizer.texts_to_sequences(X)
    X = pad_sequences(X, maxlen=sequence_length)
    return X

In [130]:
X_crawl = pre_process_comment_array(crawl_X)

In [167]:
XX_crawl = make_featues_comment_array(X_crawl, tokenizer)

In [168]:
XX_crawl

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 7]])

In [134]:
# Make predictions
predictions = lstmModel.predict(XX_crawl)

predictions

# # Add the predicted results to the DataFrame
# data_crawl['predicted_result'] = [int(prediction > 0.5) for prediction in predictions]

# data_crawl[['Review Text', 'predicted_result']]

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


array([[0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.99997157],
       [0.9536588 ],
       [0.99999994],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.99997157],
       [0.9536588 ],
       [0.99999994],
       [0.9536588 ],
       [0.9536588 ],
       [0.9999998 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9999998 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.99999994],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.99999994],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.99999994],
       [0.9536588 ],
       [0.9536588 ],
       [0.953

TEST OLD DATA

In [165]:
PATH_TEST_V1 = os.path.join(DIR_DATASET, 'core-data\\test_ver1.csv')
data_test_v1 = pd.read_csv(PATH_TEST_V1)

In [170]:
dt1_X = data_test_v1.iloc[:, 4:5]
# y_train = data_train.iloc[:, 1:2]
dt1_X = dt1_X.dropna()

X_dt1 = pre_process_comment_array(crawl_X)
XX_dt1 = make_featues_comment_array(X_dt1, tokenizer)

In [174]:
# Make predictions
predictions = lstmModel.predict(XX_crawl)

predictions

# # Add the predicted results to the DataFrame
# data_crawl['predicted_result'] = [int(prediction > 0.5) for prediction in predictions]

# data_crawl[['Review Text', 'predicted_result']]

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


array([[0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9937124 ],
       [0.757134  ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9937124 ],
       [0.757134  ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9536588 ],
       [0.9865527 ],
       [0.9536588 ],
       [0.953